In [ ]:
train_path = '/content/OpenEnded_mscoco_train2014_questions.json'
val_path = '/content/OpenEnded_mscoco_val2014_questions.json'

In [ ]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 2.8MB/s 


In [ ]:
import torch

def embed(text, tokenizer, model):
  # Load pre-trained model tokenizer (vocabulary)
  #tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

  marked_text = "[CLS] " + text + " [SEP]"

  # Tokenize our sentence with the BERT tokenizer.
  tokenized_text = tokenizer.tokenize(marked_text)

  # Map the token strings to their vocabulary indeces.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  # Mark each of the 22 tokens as belonging to sentence "1".
  segments_ids = [1] * len(tokenized_text)

  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  # Load pre-trained model (weights)
  #model = BertModel.from_pretrained('bert-base-uncased')

  # Put the model in "evaluation" mode, meaning feed-forward operation.
  model.eval()

  # Predict hidden states features for each layer
  with torch.no_grad():
      encoded_layers, _ = model(tokens_tensor, segments_tensors)
  # `encoded_layers` has shape [12 x 1 x 22 x 768]

  # `token_vecs` is a tensor with shape [22 x 768]
  token_vecs = encoded_layers[11][0]

  # Calculate the average of all 22 token vectors.
  sentence_embedding = torch.mean(token_vecs, dim=0)

  return sentence_embedding

In [ ]:
import json
with open(train_path, 'r') as fd:
            questions_json = json.load(fd)

In [ ]:
import numpy as np
import h5py
from pytorch_pretrained_bert import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

h5_file = h5py.File('/content/drive/My Drive/Colab Notebooks/qa_path/train_q_real.h5', 'w')
data = h5_file.create_dataset('data', shape=(248349, 768), dtype=np.float64, fillvalue=0)

def prepare_questions(questions_json):
    questions = [q['question'] for q in questions_json['questions']]
    #print(len(questions))
    counter=0
    for question in questions:
        counter+=1
        if(counter%1000 == 0):
          print('Processed '+(str)(counter)+' questions')
        question = question.lower()[:-1]
        data[counter-1] = embed(question, tokenizer, model)

prepare_questions(questions_json)
h5_file.close()
questions = list(prepare_questions(questions_json))


Processed 1000 questions
Processed 2000 questions


KeyboardInterrupt: ignored

In [ ]:
data = h5py.File('/content/drive/My Drive/Colab Notebooks/qa_path/val_q.h5', 'r')
ar = data['data'][1]
#print(ar)
t = torch.tensor(ar)
#print(t)
#print(t.shape)

In [ ]:
qs = [ torch.tensor(q, dtype=torch.float64) for q in data['data'] ]
print(len(qs))
print(qs[0].shape)
print(qs[0][:5])
print(qs[1105][:5])
print(qs[29997][:5])
print(qs[29998][:5])
print(qs[29999][:5])

30000
torch.Size([768])
tensor([-0.0660, -0.3230, -0.0460, -0.0660,  0.4901], dtype=torch.float64)
tensor([-0.1644,  0.1507, -0.0917,  0.1274,  0.1025], dtype=torch.float64)
tensor([ 0.2105,  0.1708, -0.0801,  0.0617,  0.7281], dtype=torch.float64)
tensor([ 0.5266,  0.1481,  0.1471, -0.3032,  0.0594], dtype=torch.float64)
tensor([-0.1489, -0.1921, -0.0801,  0.0474,  0.5805], dtype=torch.float64)


In [ ]:
print(questions[2])
tensor1 = (embed(questions[2], tokenizer, model))
tensor2 = (embed(questions[3], tokenizer, model))

for i in range(768):
    print('{} {}'.format(tensor1[i],tensor2[i]))

where is the man sitting
-0.03696434944868088 0.14837650954723358
0.015437464229762554 0.051264435052871704
0.06961556524038315 0.05781183019280434
0.12220525741577148 -0.059995394200086594
0.47545671463012695 0.6131995320320129
-0.030782027170062065 0.22683154046535492
-0.02510000951588154 -0.1252213567495346
0.48267969489097595 0.3875361382961273
-0.21593043208122253 -0.18538688123226166
-0.17462347447872162 -0.2633647620677948
0.06435434520244598 0.4358271062374115
-0.039950426667928696 -0.08932927250862122
0.013003461062908173 -0.13226664066314697
0.14126846194267273 0.15925373136997223
-0.1895371526479721 -0.16804616153240204
0.4259328544139862 0.35926368832588196
0.23532511293888092 0.3785689175128937
-0.10232572257518768 0.15112869441509247
0.023811692371964455 0.05412256345152855
0.4691171646118164 0.22333437204360962
0.055501848459243774 0.2949240207672119
0.15813598036766052 0.027374267578125
-0.06388293951749802 0.038222070783376694
-0.24216338992118835 0.024987583979964256
